In [6]:
import re

def parse_function(term):
    match = re.match(r'(\w+)\((.*)\)', term)
    if match:
        name, params = match.group(1), match.group(2).split(',')
        return name, [p.strip() for p in params]
    return None

def unify_var(var, value, theta):
    if var in theta:
        return unify(theta[var], value, theta)
    elif value in theta:
        return unify(var, theta[value], theta)
    else:
        theta[var] = value
    return theta

def is_variable(term):
    return isinstance(term, str) and len(term) == 1 and term.isalpha()

def unify(term1, term2, theta={}):
    term1, term2 = theta.get(term1, term1), theta.get(term2, term2)
    
    if term1 == term2:
        return theta
    elif is_variable(term1):
        return unify_var(term1, term2, theta)
    elif is_variable(term2):
        return unify_var(term2, term1, theta)
    elif (func1 := parse_function(term1)) and (func2 := parse_function(term2)):
        if func1[0] == func2[0] and len(func1[1]) == len(func2[1]):
            for p1, p2 in zip(func1[1], func2[1]):
                theta = unify(p1, p2, theta)
            return theta
    else:
        theta[term1] = term2
    return theta

def unify_sentences(sentence1, sentence2):
    func1, func2 = parse_function(sentence1), parse_function(sentence2)
    if func1 and func2 and func1[0] == func2[0] and len(func1[1]) == len(func2[1]):
        theta = {}
        for p1, p2 in zip(func1[1], func2[1]):
            theta = unify(p1, p2, theta)
        return {k: theta[k] for k in sorted(theta)}  # Sort for consistent output
    return {}

# Testing the implementation with the required examples
print("Unifying Prime(13) and Prime(y):", unify_sentences("Prime(13)", "Prime(y)"))  # Θ = {y/13}
print("Unifying Knows(John, x) and Knows(y, Mother(y)):", unify_sentences("Knows(John, x)", "Knows(y, Mother(y))"))  # Θ = {y/John, x/Mother(John)}
print("Unifying pro(b, X, f(g(Z))) and pro(Z, f(Y), f(Y)):", unify_sentences("pro(b, X, f(g(Z)))", "pro(Z, f(Y), f(Y))"))  # Θ = {Z/b, X/f(Y), Y/g(b)}
print("Unifying Quick(a, g(x, a), f(y)) and Quick(a, g(f(b), a), x):", unify_sentences("Quick(a, g(x, a), f(y))", "Quick(a, g(f(b), a), x)"))  # Θ = {a/a, x/f(b), y/b}


Unifying Prime(13) and Prime(y): {'y': '13'}
Unifying Knows(John, x) and Knows(y, Mother(y)): {'x': 'Mother(y)', 'y': 'John'}
Unifying pro(b, X, f(g(Z))) and pro(Z, f(Y), f(Y)): {'X': 'f(Y)', 'Y': 'g(Z)', 'b': 'Z'}
Unifying Quick(a, g(x, a), f(y)) and Quick(a, g(f(b), a), x): {'g(x': 'g(f(b)', 'x': 'f(y)'}
